In [1]:
import pandas as pd 
import numpy

In [2]:
data = pd.read_json('image_review_all.json', lines=True)

/home/ubuntu/.conda/envs/cse158a2/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:1196: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


In [5]:
data['user_id'] = data['user_id'].astype(str)

In [6]:
dataTrain = data[:int(len(data)*.9)]
dataTest = data[int(len(data)*.9):]

In [7]:
dataTrain.head()

,business_id,user_id,rating,review_text,pics
0,605730f68cd0e3d69a52284b,1.1389089287259985e+20,4,We came for a birthday brunch and this place i...,[{'id': 'AF1QipPrls2G30PS3tyC55KBxUrKgy3ER0AB5...
1,605730f68cd0e3d69a52284b,1.00158382312396e+20,5,"Cool place to hang out, have drinks. There is...",[{'id': 'AF1QipPj8FEVZrdpTZmRdjoOtzQyfGYSwJ0Ub...
2,605730f68cd0e3d69a52284b,1.1349516171898012e+20,2,This place doesn’t rock the senses when it com...,[{'id': 'AF1QipOP5poDTRQ4XXIM11buv5x9Ae-BNXwcq...
3,605730f68cd0e3d69a52284b,1.1125954440107526e+20,3,Pretty cool place to hang out with friends and...,[{'id': 'AF1QipMq84NEaCMdQtfrELbudJdQbTILp_5I_...
4,605730f68cd0e3d69a52284b,1.0428546327506392e+20,5,We saw this place the other day when we had re...,[{'id': 'AF1QipOTCVzPHHWTiQO_2UZNVeG5KMbcki6d5...


In [9]:
# Calculate the global average rating
global_average = dataTrain['rating'].mean()

# Calculate the user-specific average ratings
user_average = dataTrain.groupby('user_id')['rating'].mean()

# Load the pairs for prediction into a dataframe
pairs_df = dataTest[['business_id', 'user_id']]

In [10]:
pairs_df = pairs_df.assign(user_average=pairs_df['user_id'].map(user_average))

# Replace NaN values (users not in training data) with the global average
pairs_df.loc[pairs_df['user_average'].isna(), 'user_average'] = global_average

# Safely assign the predicted rating column
pairs_df = pairs_df.assign(predicted_rating=pairs_df['user_average'])

In [17]:
pairs_df.head()

,business_id,user_id,user_average,predicted_rating
1338972,60423c5e2e57ebdea29c8d6d,1.1615514791549198e+20,3.500000,3.500000
1338973,60423c5e2e57ebdea29c8d6d,1.0150434207857227e+20,4.666667,4.666667
1338974,60423c5e2e57ebdea29c8d6d,1.1520009869201855e+20,4.450040,4.450040
1338975,60423c5e2e57ebdea29c8d6d,1.112238683133585e+20,5.000000,5.000000
1338976,60423c5e2e57ebdea29c8d6d,1.129038314252033e+20,4.450040,4.450040


In [19]:
pred = pairs_df[['predicted_rating']].round()

In [21]:
pred['actual'] = dataTest['rating']

In [25]:
pred.shape

(148775, 2)

In [26]:
sum(pred['predicted_rating'] == pred['actual'])/148775

0.38359939505965385